<h1>1.读取认证信息和租户信息</h1>
认证信息不需要在每次调用接口时都去通过登录重新获取，认证信息的有效时间是7天，因此在7天内都可以重复使用，可以直接读取user_info.json中保存的认证信息。

In [ ]:
import json

with open('user_info.json', 'r', encoding='utf-8') as f:
    payload = json.load(f)

token=payload["token"]
tenantId=payload["tenantId"]
print("认证信息：",token)
print("租户信息：",tenantId)

<h1>2.通过mqtt推送实测数据</h1>
IP地址：172.23.21.148</br>
端口号：1883</br>
topic：v1/devices/me/telemetry</br>
内容格式：{"ts":1634712287000,"values":{"temperature":26, "humidity":87}}</br>
设备令牌：用于连接，如：client.username_pw_set("KwWfsfHMN9f6UVt35tFB","")中的KwWfsfHMN9f6UVt35tFB，目前只能手动去thingboard中复制

In [ ]:
import time
import json
import random
from paho.mqtt import client as mqtt_client

broker = '172.23.21.148'  # mqtt代理服务器地址
port = 1883
keepalive = 60     # 与代理通信之间允许的最长时间段（以秒为单位）              
topic = "v1/devices/me/telemetry"  # 消息主题
client_id = f'python-mqtt-pub-{random.randint(0, 1000)}'  # 客户端id不能重复
def connect_mqtt():
    '''连接mqtt代理服务器'''
    def on_connect(client, userdata, flags, rc):
        '''连接回调函数'''
        # 响应状态码为0表示连接成功
        if rc == 0:
            print("Connected to MQTT OK!")
        else:
            print("Failed to connect, return code %d\n", rc)
    # 连接mqtt代理服务器，并获取连接引用
    client = mqtt_client.Client(client_id)
    client.username_pw_set("KwWfsfHMN9f6UVt35tFB","")
    client.on_connect = on_connect
    client.connect(broker, port, keepalive)
    return client

def publish(client):
    '''发布消息'''
    msg = "{\"ts\":1634712287000,\"values\":{\"temperature\":26, \"humidity\":87}}"
    result = client.publish(topic, msg)
    status = result[0]
    if status == 0:
        print(f"Send `{msg}` to topic `{topic}`")
    else:
        print(f"Failed to send message to topic {topic}")

def run():
    '''运行发布者'''
    client = connect_mqtt()
    # 运行一个线程来自动调用loop()处理网络事件, 非阻塞
    client.loop_start()
    publish(client)

run()



<h1>3.获取实测数据</h1>
推送实测数据有三种方式，包括http、opcua、mqtt，分别对应了不同的推送方式，但获取数据的接口都是同一个。也就是如下所示接口</br>
在获取数据时需要用到设备ID/指标名称，因此需要先获取设备信息，解析出deviceId和Indicator。

In [ ]:
from dhicn_iot_service import *

def find_key(key, array):
    if isinstance(array, dict):
        if array['key']==key:
            return array['value']
        else:
             return None
    elif isinstance(array, list):
        for item in array:
            result = find_key(key, item)
            if result is not None:
                return result


configuration=Configuration.get_default_copy()
configuration.host="https://online-products.dhichina.cn/gateway/iot-service" #服务地址
api_client=ApiClient(configuration)
api_client.set_default_header('Authorization',token)
api_client.set_default_header('tenantId',tenantId)

#获取设备信息
device_instance=DeviceApi(api_client)
param={
    'page_index':1,
    'page_size':1000
}
response = device_instance.api_v1_iot_device_list_get(**param)
attributes=response["data"]["list"][0]["attributes"]
indicators = find_key("indicators",attributes)

print(indicators)
indicators=json.loads(indicators)
print(indicators[0]["Indicator"])
print(indicators[1]["Indicator"])

#构建参数
timeseries_input=[{
        "deviceId": response["data"]["list"][0]["id"],
        "endTs": "2023-08-04T18:13:27",
        "keys": [indicators[0]["Indicator"],indicators[0]["Indicator"]],
        "startTs": "2023-08-03T16:13:27",
        "orderBy": "asc"
    },
    {
        "deviceId": response["data"]["list"][1]["id"],
        "endTs": "2023-08-04T18:13:27",
        "keys": [indicators[0]["Indicator"],indicators[0]["Indicator"]],
        "startTs": "2023-08-03T16:13:27",
        "orderBy": "asc"
    }
]
telemetry_instance=TelemetryApi(api_client)
response=telemetry_instance.api_v2_iot_timeseries_batch_post(timeseries_input=timeseries_input)
print("result:", response)
if response["status"] == False:
    os._exit()